<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Random Search로 KNN 알고리즘 모델을 튜닝합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [61]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [62]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [63]:
# 상위 몇 개 행 확인
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [64]:
# 기초통계량 확인
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [65]:
# target 확인
target = 'medv'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 정규화**

In [66]:
# 정규화
x = (x - x.min()) / (x.max() - x.min())

**3) 학습용, 평가용 데이터 분리**

In [67]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                    random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 모델 선언시 n_neighbors=5를 지정합니다.

In [68]:
# 불러오기
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [69]:
# 선언하기
model = KNeighborsRegressor(n_neighbors=5)

In [70]:
# 학습하기
model.fit(x_train, y_train)

KNeighborsRegressor()

In [71]:
# 예측하기
y_pred = model.predict(x_test)

In [72]:
# 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 3.0527631578947365
R2-Score: 0.7860470863341451


In [73]:
# 파라미터 확인
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

# 5.성능 튜닝


- Random Search로 튜닝을 진행합니다. 

**1) 모델링 & 튜닝**

In [74]:
a = list(range(3, 51)) # [3, 4, 5, ... , 50]
a

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

In [75]:
# 파라미터 선언
  # n_neighbors: range(3, 51)
rand_param = {'n_neighbors':range(3, 51)}

# 확인
rand_param

{'n_neighbors': range(3, 51)}

In [76]:
# 불러오기
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV

In [77]:
# 선언하기
model_knn = KNeighborsRegressor()

# Random Search 선언
  # cv=10
  # n_iter=30
  # scoring='r2'
model = RandomizedSearchCV(model_knn, # 기본 모델 이름
                          rand_param, # 앞에서 선언한 튜닝용 파라미터 변수
                          cv=10,      # K-Fold Cross Validation(default=5)
                          n_iter=30,  # Random하게 시도할 횟수(default=10)
                          scoring='r2', # 평가 방법
                          n_jobs=-1) # cpu성능 좋아지게 병렬처리

In [78]:
# 학습하기
model.fit(x_train, y_train)

RandomizedSearchCV(cv=10, estimator=KNeighborsRegressor(), n_iter=30, n_jobs=-1,
                   param_distributions={'n_neighbors': range(3, 51)},
                   scoring='r2')

**2) 결과 확인**

In [79]:
model.cv_results_

{'mean_fit_time': array([0.00793951, 0.0069124 , 0.00856915, 0.00574813, 0.00571117,
        0.00631735, 0.00587482, 0.00600266, 0.00598829, 0.00509896,
        0.00499678, 0.00514708, 0.00569322, 0.00710931, 0.00613449,
        0.00609691, 0.00612512, 0.00693393, 0.00772812, 0.00563509,
        0.00600319, 0.00597544, 0.00638945, 0.00616333, 0.00565495,
        0.00612073, 0.0061141 , 0.00595298, 0.00644941, 0.00562882]),
 'std_fit_time': array([0.00203704, 0.00154384, 0.00519421, 0.00076332, 0.00070776,
        0.00181736, 0.00096172, 0.00097066, 0.00107548, 0.00043271,
        0.00054552, 0.00036286, 0.00062383, 0.00339484, 0.00090458,
        0.00118936, 0.00159767, 0.00265449, 0.00341639, 0.00074618,
        0.00105194, 0.00090917, 0.0013525 , 0.00245181, 0.00042929,
        0.00148012, 0.00111658, 0.00104449, 0.00168206, 0.00052767]),
 'mean_score_time': array([0.00621498, 0.00691903, 0.00648613, 0.00565634, 0.00565674,
        0.00573716, 0.00531664, 0.00548267, 0.00517695, 0.00

In [80]:
# mean_test_score 확인
model.cv_results_['mean_test_score']

array([0.52389813, 0.47517792, 0.5899938 , 0.50868836, 0.43395936,
       0.63644506, 0.58122107, 0.50091229, 0.59420654, 0.54705204,
       0.60325605, 0.67869753, 0.58607069, 0.56894534, 0.69316234,
       0.55715091, 0.4709524 , 0.54007307, 0.55235359, 0.56533742,
       0.43886195, 0.60445655, 0.51481402, 0.56125186, 0.59231767,
       0.46495258, 0.59808653, 0.48629809, 0.44380257, 0.46188807])

In [81]:
# 최적 파라미터
model.best_params_

{'n_neighbors': 3}

In [82]:
# 최고 성능
model.best_score_

0.693162339471915

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [83]:
# 예측하기
y_pred = model.predict(x_test)

In [84]:
# 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 2.7524122807017544
R2-Score: 0.8292574644994108
